This notebook showcases the API calls made to get the source data for projects and issuances from the worlds two biggest registires that are manily used in Kenya, which are the gold standard and verra registyries.
here are the links to the registries:

Verra: https://registry.verra.org/app/search/VCS

Goldstandard: https://registry.goldstandard.org/projects?q=&page=1

# Gold standard

In [ ]:
# gold registry projects download all pages
import requests
import pandas as pd
import time
from google.colab import files

# Function to fetch data from the API with retry logic
def fetch_data_from_api(page=1, size=25):
    url = f"https://public-api.goldstandard.org/projects?query=&page={page}&size={size}&sortColumn=&sortDirection="

    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "en-GB,en;q=0.9",
        "origin": "https://registry.goldstandard.org",
        "referer": "https://registry.goldstandard.org/",
        "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
    }

    for attempt in range(5):  
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:  
                print(f"Rate limit exceeded. Attempt {attempt + 1}/5. Retrying in {2 ** attempt} seconds...")
                time.sleep(2 ** attempt)  
            else:
                raise e
    raise Exception("Failed to fetch data after multiple attempts")

# Function to process the JSON data
def process_data(json_data):
    if isinstance(json_data, dict) and 'data' in json_data:
        projects = json_data['data']  # Adjust based on actual structure
    else:
        projects = json_data  # Handle case where JSON is a list directly

    return projects

# Fetch and process data
all_data = []
page = 1
size = 25  # You can adjust the batch size if needed

while True:
    json_data = fetch_data_from_api(page=page, size=size)
    data = process_data(json_data)

    if not data:
        break  # 

    all_data.extend(data)
    page += 1  # Move to the next page

    # Optional: A
    time.sleep(1)

# Convert the list of all data to a DataFrame
df = pd.DataFrame(all_data)

The issuances for Gold standard follow a similar pattern to the proejects the URL has been adjusted. 

In [ ]:
# credit
import requests
import pandas as pd
import time

# Function to fetch data from the API with retries and rate limit handling
def fetch_data_from_api(page=1, size=25):
    url = f"https://public-api.goldstandard.org/credits?query=&page={page}&size={size}&sortColumn=&sortDirection=&issuances=true"
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "en-GB,en;q=0.9",
        "origin": "https://registry.goldstandard.org",
        "referer": "https://registry.goldstandard.org/",
        "sec-ch-ua": "\"Chromium\";v=\"128\", \"Not;A=Brand\";v=\"24\", \"Google Chrome\";v=\"128\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
    }

    for attempt in range(5):  # Retry up to 5 times
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error if the request failed
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:  # Rate limit error
                print(f"Rate limit exceeded. Attempt {attempt + 1}/5. Retrying in {2 ** attempt} seconds...")
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                raise e

    raise Exception("Failed to fetch data after multiple retries due to rate limit issues.")

# Function to process the JSON data into a DataFrame
def process_data(json_data):
    credits = json_data  # Assuming the root of the response is a list
    df = pd.DataFrame(credits)
    return df

# Fetch all pages of data
#all_data = []
page = 1
size = 25

json_data = fetch_data_from_api(page, size)
df = process_data(json_data)

# Verra 

In [ ]:
# Verra projects list
import requests
import pandas as pd
import time

# Function to fetch data from the API
def fetch_data_from_api(skip=0, top=50):
    url = "https://registry.verra.org/uiapi/resource/resource/search?maxResults=2000&$count=true&$skip={}&$top={}".format(skip, top)

    headers = {
        "Accept": "application/json",
        "Accept-Language": "en-GB,en;q=0.9",
        "Connection": "keep-alive",
        "Content-Type": "application/json",
        "Origin": "https://registry.verra.org",
        "Referer": "https://registry.verra.org/app/search/VCS/All%20Projects",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
        "sec-ch-ua": "\"Chromium\";v=\"128\", \"Not;A=Brand\";v=\"24\", \"Google Chrome\";v=\"128\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\""
    }

    payload = {
        "program": "VCS",
        "regions": [
            "AFRICA", "ASIA", "EUROPE", "LATIN_AMERICA",
            "MIDDLE_EAST", "NORTH_AMERICA", "OCEANIA"
        ],
        "resourceStatuses": [
            "VCS_EX_D_CRD_PRD_VER_REQUESTED", "VCS_EX_CRD_PRD_VER_REQUESTED",
            "VCS_EX_D_CRD_PRD_REQUESTED", "VCS_EX_CRD_PRD_REQUESTED",
            "VCS_EX_INACTIVE", "VCS_EX_LATE_TO_VERIFY",
            "VCS_EX_ONHOLD", "VCS_EX_REGISTERED",
            "VCS_EX_D_REG_VER_APPR_REQUESTED", "VCS_EX_REG_VER_APPR_REQUESTED",
            "VCS_EX_D_REGISTRATION", "VCS_EX_REGISTRATION_REQUESTED",
            "VCS_EX_REJ", "VCS_EX_UNDER_DEVELOPMENT_CLD",
            "VCS_EX_UNDER_VALIDATION_CLD", "VCS_EX_UNDER_VALIDATION_OPN",
            "VCS_EX_CRED_TRANS_FRM_OTHER_PROG", "VCS_EX_D_VER_APPROVAL_REQUESTED",
            "VCS_EX_VER_APPROVAL_REQUESTED", "VCS_EX_WITHDRAWN"
        ]
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad status codes
    return response.json()

# Fetch and process all data
all_data = []
skip = 0
top = 50

while True:
    json_data = fetch_data_from_api(skip=skip, top=top)
    data = json_data.get('value', [])  # Adjust key based on API response structure
    if not data:
        break
    all_data.extend(data)
    skip += top
    time.sleep(1)  # Adding delay to avoid rate limit issues

df = pd.DataFrame(all_data)
df.info()
df.head()